In [0]:
# Databricks notebook: mount_and_list_files

# -----------------------------------
# Imports & Logging Setup
# -----------------------------------
from pyspark.sql import SparkSession
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# -----------------------------------
# Widgets (User Parameters)
# -----------------------------------
dbutils.widgets.text("storage_account_name", "arkahmanalyticsadls", "Storage Account")
dbutils.widgets.text("container_name", "bronze", "Container Name")
dbutils.widgets.text("secret_scope", "azure-secrets", "Secret Scope")

# -----------------------------------
# Parameter Retrieval
# -----------------------------------
storage_account_name = dbutils.widgets.get("storage_account_name")
container_name = dbutils.widgets.get("container_name")
secret_scope = dbutils.widgets.get("secret_scope")

mount_point = f"/mnt/{storage_account_name}/{container_name}"
blob_url = f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net"
secret_key = f"{storage_account_name}-access-key"

# -----------------------------------
# Mount Function
# -----------------------------------
def mount_container(mount_point: str, blob_url: str, storage_account_name: str, storage_account_key: str):
    try:
        if any(mount.mountPoint == mount_point for mount in dbutils.fs.mounts()):
            logger.info(f"{mount_point} is already mounted.")
        else:
            dbutils.fs.mount(
                source=blob_url,
                mount_point=mount_point,
                extra_configs={f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net": storage_account_key}
            )
            logger.info(f"✅ Mounted {mount_point} successfully.")
    except Exception as e:
        logger.error(f"❌ Failed to mount: {e}")
        raise

# -----------------------------------
# File Listing Function
# -----------------------------------
def list_files_in_container(mount_point: str):
    try:
        files = dbutils.fs.ls(mount_point)
        if not files:
            logger.info(f"📂 The directory {mount_point} is empty.")
        else:
            logger.info(f"📁 Files in {mount_point}:")
            for file in files:
                logger.info(f" - {file.name}")
        return files
    except Exception as e:
        logger.error(f"❌ Error listing files: {e}")
        return []

# -----------------------------------
# Main Logic
# -----------------------------------
try:
    storage_account_key = dbutils.secrets.get(scope=secret_scope, key=secret_key)
    mount_container(mount_point, blob_url, storage_account_name, storage_account_key)
    _ = list_files_in_container(mount_point)
except Exception as e:
    logger.error(f"Critical error: {e}")
